# NICE for calibration

In [1]:
import sys
import random
sys.path.append('..')

import numpy as np
import matplotlib.pyplot as plt
from scipy.special import softmax
from sklearn.isotonic import IsotonicRegression
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Activation, Input
from tensorflow.keras.optimizers import Adam, SGD

from utils.ops import onehot_encode, optim_temperature, detection_log_likelihood_ratios
from utils.metrics import neg_log_likelihood
from utils.visualization import plot_pdf_simplex, plot_prob_simplex, reliability_plot, ECE_plot
from flows.nice import AddCouplingLayer, MLP
from calibrators import PAVCalibrator, MLRCalibrator

ImportError: cannot import name 'plot_pdf_simplex'

In [ ]:
%matplotlib inline
np.random.seed(2)

## Define NICE flow

In [ ]:
input_dim = 3
activation='relu'
hidden_size = [3, 3]

### Coupling functions

In [ ]:
m1 = MLP(input_dim//2, input_dim - input_dim//2, activation=activation, hidden_size=hidden_size)
m2 = MLP(input_dim - input_dim//2, input_dim//2, activation=activation, hidden_size=hidden_size)
m3 = MLP(input_dim//2, input_dim - input_dim//2, activation=activation, hidden_size=hidden_size)
m4 = MLP(input_dim - input_dim//2, input_dim//2, activation=activation, hidden_size=hidden_size)

### Forward flow 

In [ ]:
inp = Input(shape=(input_dim,))

x = AddCouplingLayer(m1)(inp)
x = AddCouplingLayer(m2, mode='even')(x)
x = AddCouplingLayer(m3)(x)
x = AddCouplingLayer(m4, mode='even')(x)

nice_flow = Model(inputs=inp, outputs=x)

### Inverse flow 

In [ ]:
inv_inp = Input(shape=(input_dim,))

x = AddCouplingLayer(m4, mode='even', inverse=True)(inv_inp)
x = AddCouplingLayer(m3, inverse=True)(x)
x = AddCouplingLayer(m2, mode='even', inverse=True)(x)
x = AddCouplingLayer(m1, inverse=True)(x)

inv_nice_flow = Model(inputs=inv_inp, outputs=x)

### Trainable model

In [ ]:
# Softmax output layer
y = Activation('softmax')(nice_flow.output)

nice_calibrator = Model(inputs=nice_flow.input, outputs=y)
nice_calibrator.summary()

## Comparison against other calibration methods

### Generate fake target disitribution and simulated predictions

In [ ]:
n_samples = 1500

In [ ]:
target = np.array(random.choices(range(3), k=n_samples))
one_hot = onehot_encode(target)

In [ ]:
# Shift
offset = np.zeros((n_samples, 3))
offset[:, 1:] = 0.3

# Twist
twisting = np.zeros((n_samples, 3))
twisting[np.arange(n_samples), target-1] = 0.7 + np.random.randn(n_samples)*0.1

snt_logits = 0.8 * (one_hot + twisting + np.random.randn(n_samples, 3)*0.3) + offset
snt_probs = softmax(snt_logits, axis=1)

nll = neg_log_likelihood(snt_probs, target)

print("Negative log-likelihood of the classifier predictions: {:.5f}".format(nll))

In [ ]:
fig, ax_arr = plt.subplots(1, 2, figsize=(17, 6))
ax_arr[0] = plot_prob_simplex(snt_probs, target=target, ax=ax_arr[0], title='Simulated classifier predictions', fontsize=12);
ax_arr[1] = plot_pdf_simplex(snt_probs, ax=ax_arr[1], title='Estimated output classifier PDF', fontsize=12);
plt.show()

### Apply temperature scaling 

In [ ]:
# Temp scaling
T = optim_temperature(snt_logits, target)
temp_probs = softmax(snt_logits/T, axis=1)

nll_temp = neg_log_likelihood(temp_probs, target)

print("Negative log-likelihood after calibration with temp-scaling: {:.3f}".format(nll_temp))

### Calibrate with PAV extended to multiclass(normalization)

In [ ]:
pav_cal = PAVCalibrator(snt_logits, target)

In [ ]:
pav_probs = pav_cal.predict(snt_logits)
nll_pav = neg_log_likelihood(pav_probs, target)
print("Negative log-likelihood after calibration with PAV: {:.3f}".format(nll_pav))

### Calibrate with Multiclass Logistic Regression 

In [ ]:
mlr_cal = MLRCalibrator(snt_logits, target)

In [ ]:
mlr_probs = mlr_cal.predict(snt_logits)
nll_mlr = neg_log_likelihood(mlr_probs, target)
print("Negative log-likelihood after calibration with mlr: {:.3f}".format(nll_mlr))

### Calibrate with NICE flow

In [ ]:
nice_calibrator.compile(optimizer=SGD(1e-3), loss='categorical_crossentropy')

In [ ]:
h = nice_calibrator.fit(snt_logits, one_hot, epochs=10000, batch_size=100, verbose=0)

# Plot training NLL
plt.plot(h.history['loss'])
plt.title('NICE-flow NLL')
plt.ylabel('NLL')
plt.xlabel('Epoch')
plt.show()

In [ ]:
nice_probs = nice_calibrator.predict(snt_logits, batch_size=100)

nll_nice = neg_log_likelihood(nice_probs, target)

print("Negative log-likelihood after calibration with NICE: {:.3f}".format(nll_nice))

### Visualizing results

In [ ]:
fig, ax_arr = plt.subplots(3, 2, figsize=(16, 22), gridspec_kw={'hspace': 0.5})
ax_arr[0, 0] = plot_pdf_simplex(snt_probs, ax=ax_arr[0, 0],
                             title='Uncalibrated classifier output.\n NLL: {:.3f}'.format(nll), fontsize=12);
ax_arr[0, 1] = plot_pdf_simplex(temp_probs, ax=ax_arr[0, 1],
                             title='Calibrated using temperature scaling.\n NLL: {:.3f}'.format(nll_temp), fontsize=12);
ax_arr[1, 0] = plot_pdf_simplex(pav_probs, ax=ax_arr[1, 0],
                             title='Calibrated using PAV.\n NLL: {:.3f}'.format(nll_pav), fontsize=12);
ax_arr[1, 1] = plot_pdf_simplex(mlr_probs, ax=ax_arr[1, 1],
                             title='Calibrated using MLR.\n NLL: {:.3f}'.format(nll_mlr), fontsize=12);
ax_arr[2, 0] = plot_pdf_simplex(nice_probs, ax=ax_arr[2, 0],
                             title='Calibrated using NICE flow.\n NLL: {:.3f}'.format(nll_nice), fontsize=12);

ax_arr[2, 1] = reliability_plot([snt_probs, temp_probs, pav_probs, mlr_probs, nice_probs], target, ax=ax_arr[2, 1], 
                                labels=['Uncalibrated probabilities', 'Calibrated using Temp-Scaling',
                                        'Calibrated using PAV', 'Calibrated using MLR', 'Calibrated using NICE']);

plt.show()

### ECE plots:

Define the detection problem, we have the hypothesis: $H_{t} \equiv P(\theta=\theta_0)$ and the hypothesis space is defined over $\{H_{t}, \neg H_{t}\}$, where $\neg$ is the negation operator. We evaluate for all possible values of $\theta_0 = \{0, 1, 2\}$

In [ ]:
LRs = detection_log_likelihood_ratios(snt_logits, np.zeros(3) + 1/3.)
nice_logits = nice_flow.predict(snt_logits)
LRs_cal = detection_log_likelihood_ratios(nice_logits, np.zeros(3) + 1/3.)
LRs_cal_MLR = detection_log_likelihood_ratios(np.log(mlr_probs), np.zeros(3) + 1/3.)

fig, ax_arr = plt.subplots(3, 3, figsize=(16, 18), gridspec_kw={'hspace': 0.5})


# H_t = P(theta=0)
pav_cal = IsotonicRegression(y_min=0, y_max=1).fit_transform(snt_probs[:, 0], target==0) 
pav_cal = pav_cal/(1. - pav_cal + 1e-7) + 1e-7

ax_arr[0, 0] = ECE_plot(np.exp(LRs[:, 0]), target==0, cal_ratios=np.exp(LRs_cal[:, 0]), ax=ax_arr[0, 0], title='Calibrated with NICE\n$P(\\theta=0)$')
ax_arr[0, 1] = ECE_plot(np.exp(LRs[:, 0]), target==0, cal_ratios=np.exp(LRs_cal_MLR[:, 0]), ax=ax_arr[0, 1], title='Calibrated with MLR\n$P(\\theta=0)$')
ax_arr[0, 2] = ECE_plot(np.exp(LRs[:, 0]), target==0, cal_ratios=pav_cal, ax=ax_arr[0, 2], title='Calibrated binary with PAV\n$P(\\theta=0)$')

# H_t = P(theta=1)
pav_cal = IsotonicRegression(y_min=0, y_max=1).fit_transform(snt_probs[:, 1], target==1) 
pav_cal = pav_cal/(1. - pav_cal + 1e-7) + 1e-7

ax_arr[1, 0] = ECE_plot(np.exp(LRs[:, 1]), target==1, cal_ratios=np.exp(LRs_cal[:, 1]), ax=ax_arr[1, 0], title='Calibrated with NICE\n$P(\\theta=1)$')
ax_arr[1, 1] = ECE_plot(np.exp(LRs[:, 1]), target==1, cal_ratios=np.exp(LRs_cal_MLR[:, 1]), ax=ax_arr[1, 1], title='Calibrated with MLR\n$P(\\theta=1)$')
ax_arr[1, 2] = ECE_plot(np.exp(LRs[:, 1]), target==1, cal_ratios=pav_cal, ax=ax_arr[1, 2], title='Calibrated binary with PAV\n$P(\\theta=1)$')

# H_t = P(theta=2)
pav_cal = IsotonicRegression(y_min=0, y_max=1).fit_transform(snt_probs[:, 2], target==2) 
pav_cal = pav_cal/(1. - pav_cal + 1e-7) + 1e-7

ax_arr[2, 0] = ECE_plot(np.exp(LRs[:, 2]), target==2, cal_ratios=np.exp(LRs_cal[:, 2]), ax=ax_arr[2, 0], title='Calibrated with NICE\n$P(\\theta=2)$')
ax_arr[2, 1] = ECE_plot(np.exp(LRs[:, 2]), target==2, cal_ratios=np.exp(LRs_cal_MLR[:, 2]), ax=ax_arr[2, 1], title='Calibrated with MLR\n$P(\\theta=2)$')
ax_arr[2, 2] = ECE_plot(np.exp(LRs[:, 2]), target==2, cal_ratios=pav_cal, ax=ax_arr[2, 2], title='Calibrated binary with PAV\n$P(\\theta=2)$')


plt.show()

## Check that NICE flow is an invertible transformation

In [ ]:
# Forward flow
nice_logits = nice_flow.predict(snt_logits, batch_size=100)
# Inverse flow
recons_logits = inv_nice_flow.predict(nice_logits, batch_size=100)

recons_rmse = np.sqrt(np.mean(np.square(snt_logits-recons_logits)))
print("Root mean square reconstrucction error is: {:.3E}".format(recons_rmse))

In [ ]:
fig, ax_arr = plt.subplots(2, 3, sharex='col', figsize=(20, 10), gridspec_kw={'hspace': 0, 'wspace': 0.3})
fig.suptitle('The trained NICE-flow is an invertible transformation', fontsize=20)

ax_arr[0, 0].scatter(snt_logits[:, 0], snt_logits[:, 1]);
ax_arr[1, 0].scatter(snt_logits[:, 0], snt_logits[:, 2]);

ax_arr[0, 1].scatter(nice_logits[:, 0], nice_logits[:, 1]);
ax_arr[1, 1].scatter(nice_logits[:, 0], nice_logits[:, 2]);

ax_arr[0, 2].scatter(recons_logits[:, 0], recons_logits[:, 1]);
ax_arr[1, 2].scatter(recons_logits[:, 0], recons_logits[:, 2]);

titles = [
    'Classifier output logits', 
    'Transformed logits', 
    'Reconstructed logits\n Reconstruccion error: {:.3E}'.format(recons_rmse)
]
y_labels = ['Z($\\theta$=1)', 'Z($\\theta$=2)']

for ax, title in zip(ax_arr[0], titles):
    ax.set_title(title, fontsize=16)

for ax in ax_arr[-1]:
    ax.set_xlabel('Z($\\theta$=0)', fontsize=14)

for ax, label in zip(ax_arr[:,0], y_labels):
    ax.set_ylabel(label, fontsize=14)

# fig.tight_layout()
plt.show()